<a href="https://colab.research.google.com/github/ZainAli24/Quater_3_online_classes/blob/main/Crewai_learn_docs_online_class_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [2]:
!pip install -Uq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 38.1 MB/s

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from crewai import LLM

llm = LLM(
    model = 'gemini/gemini-2.0-flash',
)

llm.call("What is the capital of italy")

 
[2025-03-13 04:10:34][🤖 LLM CALL STARTED]: 2025-03-13 04:10:34.960203
 
[2025-03-13 04:10:35][✅ LLM CALL COMPLETED]: 2025-03-13 04:10:35.632728


'The capital of Italy is **Rome**.\n'

In [5]:
google_embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
}


In [6]:
pip install google-generativeai

In [7]:
pip install litellm

# 2. **Knowledge (Rag) :**

In [8]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = """My name is Zain, a 20-year-old student living in Dulmi Village, Gujar Khan City.
I study at PIAIC and every Sunday, I enjoy playing cricket with my brother Sameer, my cousins,
and my friends. Besides my passion for sports, I am a dedicated food enthusiast with a special love
for biryani, burgers, and sandwiches. I am equally passionate about learning and developing
AI agents—my favorite subjects being Computer Science and Math, which are deeply connected to Artificial Intelligence.
Ultimately, my goal is to create innovative AI agents. Additionally, my father, Asif, works at the post office in Gujar Khan.
"""
string_source = StringKnowledgeSource(
    content=content,
)

# Create an LLM with a temperature of 0 to ensure deterministic outputs
llm = LLM(model="gemini/gemini-1-5-flash", temperature=0)

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder,
)

result = crew.kickoff(inputs={"question": "What city does Zain live in and how old is he?"})

# **3. Memory :**

In [9]:
import os
os.environ['CREWAI_STORAGE_DIR'] = '/my_crew1'

In [10]:
from crewai import LLM

llm = LLM(
    model = 'gemini/gemini-2.0-flash',
)


In [11]:
from crewai import Crew, Process
from crewai import Agent, Task, Crew, Process, LLM
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage1.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term1/"
            )
        ),
    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),
    verbose=True,
)

In [12]:
pip install mem0ai


In [13]:
from google.colab import userdata
os.environ['MEM0_API_KEY'] =  userdata.get('MEM0_API_KEY')

# **4. Integrating Mem0 for Enhanced User Memory :**

In [14]:

import os
from crewai import Crew, Process
from crewai import Agent, Task, Crew, Process, LLM
from mem0 import MemoryClient

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = "m0-dvu4dsWnzjf6cMqkEgAedNlwOzcklDHAu0pXANKj"

# Step 1: Record preferences based on past conversation or user input
client = MemoryClient()
messages = [
    {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
    {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
    {"role": "user", "content": "I am more of a beach person than a mountain person."},
    {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
    {"role": "user", "content": "I like Airbnb more."},
]
client.add(messages, user_id="zain")

# Step 2: Create a Crew with User Memory

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    verbose=True,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"},
    },
)

crew.kickoff(inputs={"question": "What is your favorite vacation destination?"})

# **5. Tools :**

In [21]:
pip install 'crewai[tools]'

In [32]:
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [23]:
embedder = {
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }

In [24]:
!pip install -Uq langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [33]:
# from langchain_google_genai import ChatGoogleGenerativeAI

llm1=LLM(model="gemini/gemini-1.5-flash")

In [57]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)


os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True,
    llm=llm
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
    planning_llm=llm
)

# Execute tasks
crew.kickoff()

# a) Custom Tool using class method:

In [46]:

from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    student_name: str = Field(..., description="Student name")
    student_roll_no : int = Field(..., description="student id")

class PiaicStudentCard(BaseTool):
    name: str = "Piaic student card generator"
    description: str = "this function will create Piaic student card"
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, student_name: str, student_roll_no: int ) -> str:
        return f"""PIAIC student card
student name: {student_name}
student roll no: {student_roll_no}
ZAIN Agent!
"""


# b) Custom Tool using decorator method:

In [45]:
from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) -> dict | str:
    """this function search piaic student fee updates, it will required roll no of PIAIC student"""
    #database

    data = {100:'paid',
         200:'unpaid'}


    status = data.get(roll_no)

    if status:
      return {"status": status}
    else:
      return "student not found"


In [47]:

from crewai import Agent, Task, Crew

card = PiaicStudentCard()


piaic_manager = Agent(
    role="PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory="""You are a master at understanding people and their preferences.""",
    tools=[card, my_tool],
    verbose=True,
    llm=llm1
)

piaic_card_creator = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question based on final context",
    expected_output="final query answer only",
    agent=piaic_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creator],
    verbose=True
)

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student my name is ZAIN and my roll number is 200, can you create my student card."
})

print(result)


 
[2025-03-13 06:23:54][🚀 CREW 'CREW' STARTED, 6B0CB8B0-642D-45EF-B47A-008680B306DC]: 2025-03-13 06:23:54.847841
 
[2025-03-13 06:23:54][📋 TASK STARTED: YOU WILL BE RESPONSIBLE FOR ALL PIAIC RELEVANT OPERATIONS, STUDENT QUERY 'I'M PIAIC STUDENT MY NAME IS ZAIN AND MY ROLL NUMBER IS 200, CAN YOU CREATE MY STUDENT CARD.' YOU MUST BE KNOW HOW TO ANSWER HIS QUESTION BASED ON FINAL CONTEXT]: 2025-03-13 06:23:54.858591
 
[2025-03-13 06:23:54][🤖 AGENT 'PIAIC MANAGER' STARTED TASK]: 2025-03-13 06:23:54.859602
# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student my name is ZAIN and my roll number is 200, can you create my student card.' you must be know how to answer his question based on final context
 
[2025-03-13 06:23:54][🤖 LLM CALL STARTED]: 2025-03-13 06:23:54.859758
 
[2025-03-13 06:23:56][✅ LLM CALL COMPLETED]: 2025-03-13 06:23:56.597082
 
[2025-03-13 06:23:56][🤖 TOOL USAGE STARTED: 'PIAIC STUDENT CARD GENERATOR']: 2

In [43]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my name is Zain roll number is 100, can you share my fee updates?."
})

print(result)


 
[2025-03-13 06:21:50][🚀 CREW 'CREW' STARTED, 6D9D7B04-DB74-4998-B6E8-BAE494571FF3]: 2025-03-13 06:21:50.833723
 
[2025-03-13 06:21:50][📋 TASK STARTED: YOU WILL BE RESPONSIBLE FOR ALL PIAIC RELEVANT OPERATIONS, STUDENT QUERY 'I'M PIAIC STUDENT  MY NAME IS ZAIN ROLL NUMBER IS 100, CAN YOU SHARE MY FEE UPDATES?.' YOU MUST BE KNOW HOW TO ANSWER HIS QUESTION BASED ON FINAL CONTEXT]: 2025-03-13 06:21:50.846276
 
[2025-03-13 06:21:50][🤖 AGENT 'PIAIC MANAGER' STARTED TASK]: 2025-03-13 06:21:50.847381
# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my name is Zain roll number is 100, can you share my fee updates?.' you must be know how to answer his question based on final context
 
[2025-03-13 06:21:50][🤖 LLM CALL STARTED]: 2025-03-13 06:21:50.847548
 
[2025-03-13 06:21:52][✅ LLM CALL COMPLETED]: 2025-03-13 06:21:52.185799
 
[2025-03-13 06:21:52][🤖 TOOL USAGE STARTED: 'PIAIC FEE UPDATE']: 2025-03-13 06:21:52.187253


# **Advance Tools :**


In [16]:
from pydantic import BaseModel, Field

class TeacherAssignmentOutPut(BaseModel):
  score: int = Field(..., description="student assignment score")
  feedback : str = Field(..., description="teacher feedback")
  question : str = Field(..., description="question")
  answer : str = Field(..., description="python code for markdown format")

In [19]:
from crewai import Agent
from crewai_tools import CodeInterpreterTool
from crewai import Task, Crew, Process

teacher = Agent(
    role="AI teacher",
    goal="you are Agentic AI teacher you have to check python assignment code submission.",
    backstory="you have to check student python code submission.",
    tools=[CodeInterpreterTool()],
    llm=llm
)

assignment_check = Task(
    description="you are Agentic AI teacher you have to check python assignment code submission. question: '''{qestion}''' student solution : '''{solution}''' ",
    expected_output="final code is runing, assign number between 1-10",
    output_pydantic=TeacherAssignmentOutPut,
    agent=teacher # Added the missing agent here
)


crew = Crew(
    agents=[teacher],
    tasks=[assignment_check],
    process=Process.sequential,
    verbose=True,
)

question = "create function add two numbers"

solution = """
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

"""

result = crew.kickoff(inputs={
    "qestion": question,
    "solution" : solution
})

print(result)


 
[2025-03-13 04:17:45][🚀 CREW 'CREW' STARTED, FFAB2EF8-96F2-4F6A-82A5-B3287CF27EED]: 2025-03-13 04:17:45.849463
 
[2025-03-13 04:17:45][📋 TASK STARTED: YOU ARE AGENTIC AI TEACHER YOU HAVE TO CHECK PYTHON ASSIGNMENT CODE SUBMISSION. QUESTION: '''CREATE FUNCTION ADD TWO NUMBERS''' STUDENT SOLUTION : '''
DEF ADD_TWO_NUMBER(NUM1 : INT , NUM2 : INT):
  RETURN NUM1 + NUM2

''' ]: 2025-03-13 04:17:45.864389
 
[2025-03-13 04:17:45][🤖 AGENT 'AI TEACHER' STARTED TASK]: 2025-03-13 04:17:45.865580
# Agent: AI teacher
## Task: you are Agentic AI teacher you have to check python assignment code submission. question: '''create function add two numbers''' student solution : '''
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

''' 
 
[2025-03-13 04:17:45][🤖 LLM CALL STARTED]: 2025-03-13 04:17:45.865911
 
[2025-03-13 04:17:47][✅ LLM CALL COMPLETED]: 2025-03-13 04:17:47.725894
 
[2025-03-13 04:17:47][🤖 TOOL USAGE STARTED: 'CODE INTERPRETER']: 2025-03-13 04:17:47.726674
 
[2025-03-13 04

In [20]:
result.to_dict()

{'score': 9,
 'feedback': "The code is well-structured and defines a function that adds two numbers as requested. The type hints are a good addition. A minor improvement would be to add a docstring to explain the function's purpose. Good job!",
 'question': 'create function add two numbers',
 'answer': 'def add_two_number(num1 : int , num2 : int):\n  return num1 + num2'}

# **6 Process :**

In [56]:

import os
from crewai import Agent, Task, Crew, Process

# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)




# Define your agents
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research and analysis on AI and AI agents",
    backstory="You're an expert researcher, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently researching for a new client.",
    allow_delegation=False,
    llm=llm
)

writer = Agent(
    role="Senior Writer",
    goal="Create compelling content about AI and AI agents",
    backstory="You're a senior writer, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently writing content for a new client.",
    allow_delegation=False,
    llm=llm
)

# Define your task
task = Task(
    description="Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.",
    expected_output="5 bullet points, each with a paragraph and accompanying notes.",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    llm=llm
)

# Instantiate your crew with a custom manager
crew = Crew(
    agents=[researcher, writer],
    tasks=[task],
    manager_agent=manager,
    # function_calling_llm=llm1,
    process=Process.hierarchical,
    verbose=True

)

# Start the crew's work
result = crew.kickoff()
from IPython.display import Markdown

Markdown(result.raw)

 
[2025-03-13 06:49:38][🚀 CREW 'CREW' STARTED, DBC262E7-33C9-4CE1-BDB8-2B514638163E]: 2025-03-13 06:49:38.051905
 
[2025-03-13 06:49:38][📋 TASK STARTED: GENERATE A LIST OF 5 INTERESTING IDEAS FOR AN ARTICLE, THEN WRITE ONE CAPTIVATING PARAGRAPH FOR EACH IDEA THAT SHOWCASES THE POTENTIAL OF A FULL ARTICLE ON THIS TOPIC. RETURN THE LIST OF IDEAS WITH THEIR PARAGRAPHS AND YOUR NOTES.]: 2025-03-13 06:49:38.065672
 
[2025-03-13 06:49:38][🤖 AGENT 'PROJECT MANAGER' STARTED TASK]: 2025-03-13 06:49:38.067027
# Agent: Project Manager
## Task: Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.
 
[2025-03-13 06:49:38][🤖 LLM CALL STARTED]: 2025-03-13 06:49:38.067252
 
[2025-03-13 06:49:52][✅ LLM CALL COMPLETED]: 2025-03-13 06:49:52.088294
 
[2025-03-13 06:49:52][🤖 TOOL USAGE STARTED: 'DELEGATE WORK TO COWORKER']: 2025-03-

* **1. The Rise of AI-Powered Personal Assistants: Beyond Task Management**

Imagine a world where your personal assistant doesn't just remind you of appointments, but proactively analyzes your schedule, anticipates potential conflicts, and even suggests optimal travel routes based on real-time traffic and weather conditions. This is the promise of the next generation of AI-powered personal assistants, moving beyond simple task management to become intelligent partners that learn your habits, understand your preferences, and seamlessly integrate into your life, managing everything from your finances to your health. This article will explore the cutting-edge technologies driving this evolution, the companies leading the charge, and the profound implications for how we live and work.

*Notes:*
*   Target Audience: General tech enthusiasts, early adopters, and those interested in productivity and lifestyle improvement.
*   Potential Sources: Interviews with AI developers, user experience experts, and individuals using advanced AI assistants. Analysis of current AI assistant capabilities and future roadmaps.
*   Key Angle: Highlight the shift from reactive to proactive assistance and the increasing personalization of AI assistants.

* **2. AI Agents in Scientific Discovery: Accelerating Research and Innovation**

The scientific community is on the cusp of a revolution, with AI agents poised to dramatically accelerate the pace of discovery. From autonomously designing experiments and analyzing vast datasets to identifying patterns invisible to the human eye and generating novel hypotheses, AI agents are empowering researchers to tackle some of the world's most pressing challenges, including drug discovery, climate change, and materials science. This article will delve into specific examples of AI agents in action, showcasing how they are being used to unlock new insights and drive innovation across various scientific disciplines, ultimately transforming the way science is conducted.

*Notes:*
*   Target Audience: Scientists, researchers, academics, and those interested in the intersection of AI and scientific advancement.
*   Potential Sources: Interviews with researchers using AI agents, case studies of successful AI-driven discoveries, and analysis of the ethical considerations of AI in science.
*   Key Angle: Focus on the tangible impact of AI agents on scientific progress and the potential to solve complex global problems.

* **3. The AI Agent Economy: Building, Deploying, and Monetizing Autonomous Systems**

A new economic frontier is emerging, driven by the rise of AI agents capable of performing a wide range of tasks autonomously. From developing AI-powered customer service bots and creating sophisticated trading algorithms to building self-driving vehicles and managing smart homes, developers are finding innovative ways to build, deploy, and monetize these autonomous systems. This article will explore the burgeoning AI agent economy, examining the tools, platforms, and business models that are enabling developers to create and profit from AI agents, opening up exciting new opportunities for entrepreneurship and innovation.

*Notes:*
*   Target Audience: Developers, entrepreneurs, investors, and those interested in the business of AI.
*   Potential Sources: Interviews with AI agent developers, analysis of AI agent platforms and marketplaces, and case studies of successful AI agent businesses.
*   Key Angle: Highlight the economic potential of AI agents and the opportunities for developers to build and monetize autonomous systems.

* **4. AI Agents and the Future of Work: Collaboration, Augmentation, and Displacement**

The integration of AI agents into the workplace is reshaping the future of work, presenting both opportunities and challenges. While some fear job displacement, others see AI agents as powerful tools that can augment human capabilities, enabling us to focus on more creative and strategic tasks. This article will explore the evolving relationship between humans and AI agents in the workplace, examining how they are collaborating to improve productivity, enhance decision-making, and create new job roles, while also addressing the critical questions surrounding job displacement and the need for workforce retraining.

*Notes:*
*   Target Audience: Business leaders, HR professionals, employees, and those interested in the future of work.
*   Potential Sources: Interviews with companies using AI agents in the workplace, analysis of labor market trends, and expert opinions on the impact of AI on employment.
*   Key Angle: Provide a balanced perspective on the impact of AI agents on the future of work, addressing both the opportunities and challenges.

* **5. The Dark Side of AI Agents: Misinformation, Manipulation, and Malicious Use**

As AI agents become more sophisticated and autonomous, so too does the potential for their misuse. From creating convincing fake news and manipulating public opinion to launching sophisticated cyberattacks and engaging in financial fraud, malicious actors are increasingly leveraging AI agents for nefarious purposes. This article will delve into the dark side of AI agents, examining the various ways they can be used to spread misinformation, manipulate individuals, and cause harm, while also exploring the ethical considerations and potential solutions for mitigating these risks.

*Notes:*
*   Target Audience: Policymakers, cybersecurity professionals, ethicists, and those interested in the risks of AI.
*   Potential Sources: Interviews with cybersecurity experts, case studies of AI-driven attacks, and analysis of the ethical considerations of AI development.
*   Key Angle: Highlight the potential dangers of AI agents and the importance of developing safeguards to prevent their malicious use.

# Code Understanding with ChatGpt:
[Conversation with CHAT GPT](https://chatgpt.com/share/67d2858c-d0cc-8002-8f65-c8ece5cbfb7c)